<a href="https://colab.research.google.com/github/CaitLittlef/pew_acec/blob/main/acec_data_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python/GEE set-up**

In [2]:
## Initializing

!pip install geemap   
# !pip install rasterio
# !pip install geopandas
# !pip install earthpy

# # Installs geemap package
# import subprocess

# try:
#     import geemap
# except ImportError:
#     print('geemap package not installed. Installing ...')
#     subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Import packages
import os
import glob


import ee
# import folium
import geemap
import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import rasterio as rio
# import geopandas as gpd
# import earthpy as et 
from tabulate import tabulate
# from folium import plugins
from random import seed
from random import randint
from pprint import pprint # tidy printing
from datetime import datetime
from ee.data import getInfo
import pickle
import statistics as st

from ee.data import getInfo
# If all you need to do is find out what's in the container, then just print() and inspect the result in the console.
# If, for some reason, you need to use Python running in the client to manipulate whatever is in the container, then use getInfo() to get the contents of the container and assign it to a variable
# Ref: https://developers.google.com/earth-engine/guides/client_server




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Trigger GEE authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=g9DyPsgSKS3rH7jnt1dl7mSqry9uCa8e6kauDAQJUDw&tc=Nw3sGOXoO_7hYTYsypYoCPhy9CjbDrA9U-YdphinHR4&cc=3o3-YX-3oKX2taleyayW_jzOYuNDnrmCYtDASel2ky0

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh-omVm_88lhXeB_dipE8059fITXIYJ_J3Smh5nPod0h1aPqHs1qmY

Successfully saved authorization token.


In [4]:
# Mount google drive (or click in left panel)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Set directories and create new folder as source directory for R analyses
proj_dir = '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC'
data_dir = os.path.join(proj_dir, 'data')
source_dir = os.path.join(data_dir, 'source')    
working_dir = os.path.join(data_dir, 'working')  


# Create the directories if they didn't exist before
try:
      os.mkdir(source_dir)
except FileExistsError: # Found this error name by trying to run.
     print('That directory already exists. Not creating it.')

try:
      os.mkdir(working_dir)
except FileExistsError: # Found this error name by trying to run.
     print('That directory already exists. Not creating it.')


# Set this as the working directory
os.chdir(working_dir)
print("This is your working data directory:",os.getcwd())
# print(et.io.HOME)

# What's inside of the full directory? Call globe module w/in glob package
# Says look at stuff (second *) that's within the stuff (first *) within the directory.
list = glob.glob(os.path.join(data_dir,"*","*"))
list

That directory already exists. Not creating it.
That directory already exists. Not creating it.
This is your working data directory: /content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/working


['/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/BLM_National_ACEC/acec.gdb',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/BLM_National_ACEC/ACEC Designated Polygons.lyr',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/BLM_National_ACEC/ACEC_DataDictionary.pdf',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/BLM_National_ACEC/BLM_National_ACEC.xml',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/RockSpringsFO_Wyo/audiba_WY_20150818',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/RockSpringsFO_Wyo/Elk_WY_SouthWindRiver_Routes',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/RockSpringsFO_Wyo/RedDesert-LittleSandyLandscape',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/RockSpringsFO_Wyo/BigSandy',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/working/amph.tif',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/working/mamm.tif',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/working/rept.tif',
 '/content/drive/MyDrive/2Pew ACEC/Pew_ACEC/data/working/bird.tif',
 '/c

In [6]:
# Grab today
today = datetime.today().strftime('%Y-%m-%d')
print(today)

2022-06-07


# **Get AOI and indicator data**

In [8]:
# Load PADUS and retain only BLM lands; holding types are stored as separate layers.
# ref: https://www.usgs.gov/programs/gap-analysis-project/pad-us-data-manual
# Confirmed elsewhere that SHAPE_Area is indeed square meters.
sqm_in_sqkm = 1000000
padus_desig = ee.FeatureCollection("USGS/GAP/PAD-US/v20/designation").filter("Mang_Name == 'BLM'")
padus_ease = ee.FeatureCollection("USGS/GAP/PAD-US/v20/easement").filter("Mang_Name == 'BLM'")
padus_fee = ee.FeatureCollection("USGS/GAP/PAD-US/v20/fee").filter("Mang_Name == 'BLM'")
padus_proc = ee.FeatureCollection("USGS/GAP/PAD-US/v20/proclamation").filter("Mang_Name == 'BLM'")


# Combine into a single layer and set equal to 1, b/c can't clip a feauture collection.
blm = padus_desig.merge(padus_ease.merge(padus_fee.merge(padus_proc)))
blm1 = ee.Image.constant(1).toInt().clipToCollection(blm)


# Select western states; put states to retain in list then filter on that list.
keeps = ee.List(['Washington', 'Oregon', 'California', 'Idaho', 'Montana', 'Wyoming',
               'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico'])
west = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.inList('NAME', keeps))
# Create mask (val = 1) of states, as updateMask won't accept feauture collection.
west1 = ee.Image.constant(1).toInt().clipToCollection(west)
# pprint(keeps.getInfo())


# Retain only blm in western US. 
# Trial and error:
# -area for image clipping must not be an image (ie west1)
# -updateMask won't take a feauture collection (ie west)
# -featureCollections have no attribute clip, so can't clip blm; must be blm1
blm_west = blm1.clipToCollection(west).updateMask(west1)
# Alt:
# blm_west = blm1.clip(west).updateMask(west1)

# # Map it (takes LONG time to load)
# map = geemap.Map(center=[40,-100], zoom=4)
# map.addLayer(west1, name = "west", shown = True, opacity = 1)
# map.addLayer(blm_west, name = "blm_west", shown = True, opacity = 1)
# map

In [9]:
# Most indicators are already pulled from prior ACEC efforts from NAU Hub.
# Here, grab indiators on GEE and clipi to western US.
amph = ee.Image('projects/GEE_CSP/Pew-ACEC/gap_amphib_richness_habitat30m').clip(west)
bird = ee.Image('projects/GEE_CSP/Pew-ACEC/gap_bird_richness_habitat30m').clip(west)
mamm = ee.Image('projects/GEE_CSP/Pew-ACEC/gap_mammal_richness_habitat30m').clip(west)
rept = ee.Image('projects/GEE_CSP/Pew-ACEC/gap_reptile_richness_habitat30m').clip(west)



In [ ]:
climAccess = ee.Image('projects/GEE_CSP/thirty-by-thirty/aw_climate_resilience').clip(west)
climStab = ee.Image('projects/GEE_CSP/thirty-by-thirty/aw_climate_stability').clip(west)
conn = ee.Image('projects/GEE_CSP/thirty-by-thirty/conus_connectivity').clip(west)
intact = ee.Image('projects/GEE_CSP/thirty-by-thirty/conus_intactness').clip(west)

# Turn on/off
# var = climAccess
# var = climStab
# var = conn
# var = intact


# Pull min/max value
minMax = var.reduceRegion(reducer = ee.Reducer.minMax(),
                           geometry = west.geometry(),
                           scale = 270,
                           maxPixels = 1e11)
# minMax.getInfo()
# Anchor values on min and max (only one band: b1)
varNorm = var.select('b1') \
          .unitScale(minMax.get('b1_min'),minMax.get('b1_max'))
# Grab projection details for export parameters
projection = varNorm.projection().getInfo()
projection



In [ ]:
# RCMAP for annual grasses -- but I think this is already on 0-100 so scale so there may be no need to normalize
# https://code.earthengine.google.com/?scriptPath=Examples%3ADatasets%2FUSGS_NLCD_RELEASES_2019_REL_RCMAP_V4_COVER

# >1 band

var = invasive

invasive = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/RCMAP/V4/COVER') \
          .filter(ee.Filter.eq('system:index', '2016')).first() \
          .select('rangeland_annual_herbaceous') \
          .clip(west)

# print('Bands:', invasive.bandNames());

# Pull min/max value
minMax = var.reduceRegion(reducer = ee.Reducer.minMax(),
                           geometry = west.geometry(),
                           scale = 270,
                           maxPixels = 1e11)
# minMax.getInfo()
# Anchor values on min and max (of target band)
varNorm = var.select('rangeland_annual_herbaceous') \
          .unitScale(minMax.get('rangeland_annual_herbaceous_min'),minMax.get('rangeland_annual_herbaceous_max'))
# Grab projection details for export parameters
projection = varNorm.projection().getInfo()
projection


In [ ]:
# Export but change description name
task = ee.batch.Export.image.toDrive(**{
    'image': var,
    'description': 'climAccessNorm', ######## CHANGE ME ########
    'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
    'scale': 1000, #1 km2 pixels
    'crs': projection['wkt'],
    'crsTransform': projection['transform'],
    'region': west.geometry()
})
task.start()


# # Viz for climAccess example
# visClim = {
#   'min':-40,
#   'max':0,
#   'palette': ['blue', 'yellow']
# }

# visClimNorm = {
#   'min':0,
#   'max':1,
#   'palette': ['blue', 'orange']
# }

# # # Map it
# map = geemap.Map(center=[40,-100], zoom=4)
# map.addLayer(var, visClim, name = "climate access", shown = True, opacity = 1)
# map.addLayer(varNorm, visClimNorm, name = "climate acccess norm", shown = True, opacity = 1)
# map


In [ ]:
# Export to Drive

#### READ THIS ####
# https://developers.google.com/earth-engine/guides/exporting


task = ee.batch.Export.image.toDrive(**{
    'image': amph,
    'description': 'amph',
    'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
    'scale': 1000, #1 km2 pixels
    'region': west.geometry()
    # 'crs': 'epsg:42303' # This never seems to work.
})
task.start()

# This doesn't work either...
# projCrs = ee.Projection('EPSG:42303').atScale(270)
# test = amph.reduceResolution(ee.Reducer.mode()).reproject(projCrs)


task = ee.batch.Export.image.toDrive(**{
    'image': bird,
    'description': 'bird',
    'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
    'scale': 1000, #1 km2 pixels
    'region': west.geometry(),
})
task.start()


task = ee.batch.Export.image.toDrive(**{
    'image': mamm,
    'description': 'mamm',
    'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
    'scale': 1000, #1 km2 pixels
    'region': west.geometry(),
})
task.start()

task = ee.batch.Export.image.toDrive(**{
    'image': rept,
    'description': 'rept',
    'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
    'scale': 1000, #1 km2 pixels
    'region': west.geometry(),
})
task.start()

task = ee.batch.Export.image.toDrive(**{
    'image': oilgas,
    'description': 'oilgas',
    'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
    'scale': 1000, #1 km2 pixels
    'region': west.geometry(),
})
task.start()

task = ee.batch.Export.image.toDrive(**{
    'image': blm_west,
    'description': 'blm_west',
    'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
    'scale': 100, #100x100 m pixels
    'region': west.geometry(),
})
task.start()

# # Export the FeatureCollection.
# task = ee.batch.Export.table.toDrive(**{
#   'collection': ynp,
#   'description':'yellowstone',
#   'folder': 'colab_scratch', 
#   'fileFormat': 'SHP'
# })
# task.start()

In [ ]:
# task = ee.batch.Export.image.toDrive(**{
#     'image': amph,
#     'description': 'amph2',
#     'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
#     'scale': 1000, #1 km2 pixels
#     'region': west.geometry(),
#     'crs': 'epsg:4326' # WORKS!!
# })
# task.start()

task = ee.batch.Export.image.toDrive(**{
    'image': amph,
    'description': 'amph3',
    'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
    'scale': 1000, #1 km2 pixels
    'region': west.geometry(),
    'crs': 'epsg:4269' 
})
task.start()

In [ ]:
# FAIL

task = ee.batch.Export.image.toDrive(**{
    'image': blm_west,
    'description': 'blm_west',
    'folder': 'working', # even tho its wd, need to specify else ends up in gen Drive folder
    'scale': 1000, #100x100 m pixels
    'region': west.geometry(),
})
task.start()

# Fail: Error: Unable to transform geometry to pixel grid. (Error code: 3)
# Even if I do this first: blm_west = blm_west.clip(west)


In [ ]:
# # Mask land cover values so that only PADUS is retained. nb landcov is only lower 48. Not sure why but mask after clip.
# # "Updates an image's mask at all positions where the existing mask is not zero. The output image retains the metadata and footprint of the input image."
# # ^ This GEE explanation suggests that maybe there's already a mask associated with the original layer. I want that mask to NOT include non-PADUS stuff.
# lc = ee.Image("projects/GEE_CSP/Pew-ACEC/nat_landcov_l48_eslf_v3_5_2018").clip(padus).updateMask(padus1)


# # Get information about the bands as a list.
# import pprint
# pprint.pprint(lc.getInfo())
# band_names = lc.bandNames()
# print('Band names:', band_names.getInfo())  # ee.List of band names


# # # It's still so giant, try reducing.
# # lc_red = lc.reduceNeighborhood()
# # var texture = naipNDVI.reduceNeighborhood({
# #   reducer: ee.Reducer.stdDev(),
# #   kernel: ee.Kernel.circle(7),
# # });




# # visLC = {
# #   'min':3000,
# #   'max':9000,
# #   'palette': ['red','yellow','blue']
# # }

# # map = geemap.Map(center=[40,-100], zoom=4)
# # # map.addLayer(padus, {'color':'purple'}, name = "fee", shown = True, opacity = 0.5)
# # map.addLayer(lc, visLC, name = "land cover", shown = True, opacity = 1)
# # map



NameError: ignored

In [ ]:
from geemap.common import connect_postgis
# Get global human mod and clip to AOI around Castner Range defined by this retangle
bb = ee.Geometry.Rectangle(-103, 29, -109,  35)
hm = ee.ImageCollection("CSP/HM/GlobalHumanModification").first() # Select the first (only) image in the collection
hm_clip = hm.clip(bb) # Only an image can be clipped

conn = ee.Image("projects/GEE_CSP/thirty-by-thirty/conus_connectivity")
conn_clip = conn.clip(bb)

# GEE
map = geemap.Map(center=[30,-106], zoom=6)

visHM = {
  'min':0,
  'max':1,
  'palette': ['blue', 'green', 'yellow', 'red']
}

visConn = {
  'min':0,
  'max':48734,
  'palette': ['blue', 'green', 'yellow', 'red']
}

# map.addLayer(hm, visHM, 'global human mod', True, 1)
map.addLayer(conn_clip, visConn, 'Connectivity', True, 1)
# map.addLayer(bb, {'color':'purple'}, name = "BB", shown = True, opacity = 0.8)

map

In [ ]:
# task = ee.batch.Export.image.toDrive(**{
#     'image': hm_clip,
#     'description': 'hmClip',
#     'folder': 'colab_scratch', 
#     'scale': 1000, #1 km pixels
#     'region': bb
# })
# task.start()


task = ee.batch.Export.image.toDrive(**{
    'image': conn_clip,
    'description': 'connClip',
    'folder': 'colab_scratch', 
    'scale': 1000, #1 km pixels
    'region': bb
})
task.start()